In [31]:
import numpy as np
import cv2
import os
import imutils

In [32]:
def rescaleFrame(frame, scale=0.5):
    width = int(frame.shape[1] * scale)
    height = int(frame.shape[0] * scale)
    dimensions = (width, height)
    return cv2.resize(frame, dimensions, interpolation=cv2.INTER_AREA)

def calculate_brightness(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Calculate the mean brightness of the image
    brightness = np.mean(gray)

    return brightness

In [33]:
def find_rectangles_within_contour(image, contour, showProcess=False):
    
    original = image.copy()
    
    min_x, min_y = float('inf'), float('inf')
    max_x, max_y = 0, 0

    # Loop through each point in the contour
    for point in contour:
        x, y = point[0]

        # Update min and max values
        min_x = min(min_x, x)
        min_y = min(min_y, y)
        max_x = max(max_x, x)
        max_y = max(max_y, y)
        
    mask = np.zeros(image.shape, dtype=np.uint8)
    cv2.drawContours(mask, [contour], -1, (255, 255, 255), -1)
    
    # cv2.imshow("Mask", mask)
    
    masked_image = cv2.bitwise_and(image, mask)
    # cv2.imshow("Masked Image", masked_image)
    # cv2.rectangle(image, (min_x, min_y), (max_x, max_y), (0, 255, 0), 2)
    
    masked_image = cv2.cvtColor(masked_image, cv2.COLOR_BGR2GRAY)
    masked_image = cv2.GaussianBlur(masked_image, (5, 5), 0)
    # masked_image = cv2.threshold(masked_image, 0, 255, cv2.THRESH_BINARY_INV)[1]
    masked_image = cv2.Canny(masked_image, 50, 150, apertureSize = 7)
    # invert 
    # masked_image = cv2.bitwise_not(masked_image)
    
    lines = cv2.HoughLinesP(image=masked_image,rho=1.5,theta=np.pi/180, threshold=100,lines=np.array([]), minLineLength=20,maxLineGap=80)

    blank = np.zeros(image.shape, dtype=np.uint8)
    
    copy = image.copy()
    
    print(len(lines))
    
    if len(lines) < 10:
        return False
    
    if lines is not None:
        a = lines.shape[0]
        for i in range(a):
            # cv2.line(image, (lines[i][0][0], lines[i][0][1]), (lines[i][0][2], lines[i][0][3]), (0, 0, 255), 3, cv2.LINE_AA)
            cv2.line(blank, (lines[i][0][0], lines[i][0][1]), (lines[i][0][2], lines[i][0][3]), (0, 0, 255), 3, cv2.LINE_AA)
        
    if showProcess:
        cv2.imshow("Masked Image", masked_image)
        cv2.imshow("Image", image)
        cv2.imshow("Lines", blank)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    
    # mask lines with original
    blank2 = cv2.cvtColor(blank, cv2.COLOR_BGR2GRAY)
    blank2 = cv2.threshold(blank2, 0, 255, cv2.THRESH_BINARY_INV)[1]
    blank2 = cv2.bitwise_not(blank2)
    
    masked = cv2.bitwise_and(image, image, mask=blank2)
    if showProcess:
        cv2.imshow("Masked Image2", masked)
    
    
    thresh = cv2.threshold(masked_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    if showProcess:
        cv2.imshow("Thresh", thresh)
    
    thresh = cv2.erode(thresh, None, iterations=3)
    thresh = cv2.dilate(thresh, None, iterations=2)
    if showProcess:
        cv2.imshow("Morphology", thresh)
    
    x_min, x_max = -1, -1
    
    done = False
    for i in range(min_x, max_x):
        if (done):
            break
        for j in range(min_y, max_y):
            b,g,r = blank[j, i]
            if r == 255:
            # if thresh[j,i] < 10:
                x_min = i
                cv2.circle(copy, (i, j), 2, (255, 0 , 0), 3)
                print ("X_MIN", x_min)
                done = True
                break
            
    done = False
    
    for i in range(max_x, min_x, -1):
        if done:
            break
        for j in range(min_y, max_y):
            # if thresh[j,i] < 10:
            b,g,r = blank[j, i]
            if r == 255:    
                x_max = i
                cv2.circle(copy, (i, j), 2, (255, 0, 0), 3)
                print("X_MAX", x_max)
                done = True
                break
    
    if x_min == -1 or x_max == -1:
        return
    
    # contour[:, 0] = np.clip(contour[:, 0], x_min, x_max)
    for point in contour:
        x, y = point[0]

        if x < x_min:
            x = x_min
        elif x > x_max:
            x = x_max
            
        point[0] = [x, y]
            
    img = original.copy()
    
    rect = cv2.minAreaRect(contour)
    box = cv2.boxPoints(rect)
    box = np.intp(box)
    cv2.drawContours(img, [box], -1, (0, 255, 0), 3)
    
    print("DONE")
    cv2.imshow("Final Image", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    # return contour

In [34]:
def find_parallel(lines):
    max_parallel_count = 0
    best_line_index = -1
    best_parallel_indices = []

    for i in range(len(lines)):
        parallel_indices = []

        for j in range(len(lines)):
            if i == j:
                continue

            # Extract line coordinates
            x1_i, y1_i, x2_i, y2_i = lines[i][0]
            x1_j, y1_j, x2_j, y2_j = lines[j][0]

            # Check if the lines are approximately parallel
            delta_x_i = x2_i - x1_i
            delta_y_i = y2_i - y1_i
            delta_x_j = x2_j - x1_j
            delta_y_j = y2_j - y1_j

            # Avoid division by zero
            if delta_x_i == 0 or delta_x_j == 0:
                continue

            slope_i = delta_y_i / delta_x_i
            slope_j = delta_y_j / delta_x_j

            if abs(slope_i - slope_j) < 0.1:
                parallel_indices.append(j)

        if len(parallel_indices) > max_parallel_count:
            max_parallel_count = len(parallel_indices)
            best_line_index = i
            best_parallel_indices = parallel_indices

    return best_line_index, best_parallel_indices


In [35]:
def detectBarcodes(image, showProcess=False):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    blurred = cv2.medianBlur(gray, 3)

    ddepth = cv2.cv.CV_32F if imutils.is_cv2() else cv2.CV_32F
    gradX = cv2.Sobel(blurred, ddepth=ddepth, dx=1, dy=0, ksize=-1)
    gradY = cv2.Sobel(blurred, ddepth=ddepth, dx=0, dy=1, ksize=-1)
    # subtract the y-gradient from the x-gradient
    
    edgeDetection = cv2.subtract(gradX, gradY)
    edgeDetection = cv2.convertScaleAbs(edgeDetection)

    kernel = np.ones((3, 3), np.uint8)
    
    _, thresh = cv2.threshold(edgeDetection, 150, 255, cv2.THRESH_BINARY)
    erode = cv2.erode(thresh, kernel,iterations= 1)
    # Calculate the mean brightness of the image
    
    lines = cv2.HoughLinesP(image=erode,rho=1.5,theta=np.pi/180, threshold=200,lines=np.array([]), minLineLength=40, maxLineGap=20)

    blank = np.zeros(image.shape, dtype=np.uint8)
    
    copy = image.copy()
    
    if lines is None or len(lines) < 10:
        return False
    
    # print(len(lines))
    # # lines = find_parallel(lines)
    # # print("Parallel Lines: ", len(lines))
    # print (lines)
    
    best_line_index, best_parallel_indices = find_parallel(lines)

    if best_line_index != -1:
        best_line = lines[best_line_index][0]
        print(f"Best line with index {best_line_index} has {len(best_parallel_indices)} parallel lines.")
    
        # Draw the best line
        cv2.line(blank, (best_line[0], best_line[1]), (best_line[2], best_line[3]), (0, 0, 255), 3, cv2.LINE_AA)
    
        # Draw all parallel lines
        for parallel_index in best_parallel_indices:
            parallel_line = lines[parallel_index][0]
            cv2.line(blank, (parallel_line[0], parallel_line[1]), (parallel_line[2], parallel_line[3]), (0, 0, 255), 3, cv2.LINE_AA)
    else:
        print("No parallel lines found.")

        
    if showProcess:
        # cv2.imshow("Masked Image", masked_image)
        cv2.imshow("Image", image)
        cv2.imshow("Sobel", edgeDetection)
        cv2.imshow("Lines", blank)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    # imageName = image_path.split(os.path.sep)[-1]

In [36]:
folderPath = 'Images\Dataset'

# List all files in the folder
files = os.listdir(folderPath)

for file in files:
    # Check if the file is an image
    if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
        # Construct the full path to the image
        image_path = os.path.join(folderPath, file)

        # Read the image
        image = cv2.imread(image_path)

        if image.shape[0] > 800 or image.shape[1] > 1500:
            image = rescaleFrame(image)

        detectBarcodes(image, True)

Best line with index 0 has 54 parallel lines.
Best line with index 0 has 25 parallel lines.
Best line with index 0 has 16 parallel lines.
Best line with index 29 has 7 parallel lines.
Best line with index 0 has 5 parallel lines.
Best line with index 0 has 34 parallel lines.
Best line with index 16 has 1 parallel lines.
Best line with index 43 has 2 parallel lines.
Best line with index 4 has 42 parallel lines.
Best line with index 43 has 56 parallel lines.
Best line with index 0 has 43 parallel lines.
Best line with index 0 has 15 parallel lines.
Best line with index 79 has 15 parallel lines.
Best line with index 6 has 40 parallel lines.
Best line with index 41 has 32 parallel lines.
Best line with index 4 has 14 parallel lines.
Best line with index 0 has 104 parallel lines.
Best line with index 2 has 65 parallel lines.
Best line with index 1 has 16 parallel lines.
Best line with index 16 has 3 parallel lines.
Best line with index 0 has 53 parallel lines.
Best line with index 4 has 49 p